In [1]:
import pandas as pd
import numpy as np
import torch
import json
from normalizer import counter
from torch.utils.data import DataLoader


In [2]:
# All of our fields have names, but we need them in numbers, so define a dictionary to convert.
global magDict
magDict = {
    'TOTUSJH': 0,
    'TOTBSQ': 1,
    'TOTPOT': 2,
    'TOTUSJZ': 3,
    'ABSNJZH': 4,
    'SAVNCPP': 5,
    'USFLUX': 6,
    'TOTFZ': 7,
    'MEANPOT': 8,
    'EPSZ': 9,
    'SHRGT45': 10,
    'MEANSHR': 11,
    'MEANGAM': 12,
    'MEANGBT': 13,
    'MEANGBZ': 14,
    'MEANGBH': 15,
    'MEANJZH': 16,
    'TOTFY': 17,
    'MEANJZD': 18,
    'MEANALP': 19,
    'TOTFX': 20,
    'EPSY': 21,
    'EPSX': 22,
    'R_VALUE': 23,
    'RBZ_VALUE': 24,
    'RBT_VALUE': 25,
    'RBP_VALUE': 26,
    'FDIM': 27,
    'BZ_FDIM': 28,
    'BT_FDIM': 29,
    'BP_FDIM': 30,
    'PIL_LEN': 31,
    'XR_MAX': 32
}

In [3]:
# Get the data from the JSON file, then return it as a tensor of input data and a list of labels
def getDataFromJSON(path="data/train_partition1_data.json", device='cpu', earlyStop=-1):
    # I might need to refactor these arguments to allow to get the test data. Problem for another day.
    # path is the path to the files, device is where to store it (CUDA), earlyStop is how many lines to 
    # read if you don't want the entire file read.
    
    # Get the dictionary to assign names to numbers
    global magDict
    
    # This dataset is heavily skewed, so we need to get the number of each type of flare.
    # This also lets us get the number of lines in the file with a sum.
    weights = counter(path, earlyStop)
    lines = np.sum(weights)
    # Check when we want to stop - the end of the file or earlier.
    if earlyStop < 0: length = lines
    else: length = min(earlyStop, lines)
    
    # Get the file and open it. 
    file = open(path)    
    
    # Declare a tensor to hold the data, and a list to hold the labels.
    # Dimensions: 0: number of entries we want. 1: the 33 fields in the data. 2: the 60 observations in each field. 
    tnsr = torch.Tensor().new_empty((length, 33, 60), device=device)
    labels = []
    flares = {'X':0, 'M':1, 'C':2, 'B':3, 'Q':4}
        
    row = -1
    for line in file:
        # Load the line as a dictionary. Row is an integer place and v is a smaller dictionary.
        d: dict = json.loads(line)
        row += 1
        for _, v in d.items(): # we use the _ because we don't want the ID.
            if earlyStop > 0 and row >= earlyStop:
                # If we don't want the entire dataset, stop loading more than we want
                return tnsr, labels, weights
            if row % 100 == 0:
                print(f'Now loading event {row}/{length}')
            # append the label to our list
            labels.append(flares[v['label']])
            
            # Break each individual dictionary into dictionaries of observations
            # Key is the string in magDict, and timeDict is a dictionary of observations over time
            for key, timeDict in v['values'].items():
                # Turn our name string into a numeric value
                location = magDict[key]
                # Get the measurements out of the time series dictionary
                for timeStamp, measurement in timeDict.items():
                    tnsr[row][location][int(timeStamp)] = measurement
    print(f'{row} lines loaded.')
    # Close the file. I'm not a heathen                    
    file.close()
    # This might be a good place to perform some post processing, but that's a question for another day.
    # Famous last words.
    return tnsr, labels, weights



In [4]:
# This file has 77270 data points. 
%time train1, labels1, weights1 = getDataFromJSON(path="data/train_partition1_data.json", earlyStop=10000)

Now loading event 0/10000
Now loading event 100/10000
Now loading event 200/10000
Now loading event 300/10000
Now loading event 400/10000
Now loading event 500/10000
Now loading event 600/10000
Now loading event 700/10000
Now loading event 800/10000
Now loading event 900/10000
Now loading event 1000/10000
Now loading event 1100/10000
Now loading event 1200/10000
Now loading event 1300/10000
Now loading event 1400/10000
Now loading event 1500/10000
Now loading event 1600/10000
Now loading event 1700/10000
Now loading event 1800/10000
Now loading event 1900/10000
Now loading event 2000/10000
Now loading event 2100/10000
Now loading event 2200/10000
Now loading event 2300/10000
Now loading event 2400/10000
Now loading event 2500/10000
Now loading event 2600/10000
Now loading event 2700/10000
Now loading event 2800/10000
Now loading event 2900/10000
Now loading event 3000/10000
Now loading event 3100/10000
Now loading event 3200/10000
Now loading event 3300/10000
Now loading event 3400/100

In [5]:
# This file has 93767 data points. 
%time train2, labels2, weights2 = getDataFromJSON(path="data/train_partition2_data.json", earlyStop=10000)

Now loading event 0/10000
Now loading event 100/10000
Now loading event 200/10000
Now loading event 300/10000
Now loading event 400/10000
Now loading event 500/10000
Now loading event 600/10000
Now loading event 700/10000
Now loading event 800/10000
Now loading event 900/10000
Now loading event 1000/10000
Now loading event 1100/10000
Now loading event 1200/10000
Now loading event 1300/10000
Now loading event 1400/10000
Now loading event 1500/10000
Now loading event 1600/10000
Now loading event 1700/10000
Now loading event 1800/10000
Now loading event 1900/10000
Now loading event 2000/10000
Now loading event 2100/10000
Now loading event 2200/10000
Now loading event 2300/10000
Now loading event 2400/10000
Now loading event 2500/10000
Now loading event 2600/10000
Now loading event 2700/10000
Now loading event 2800/10000
Now loading event 2900/10000
Now loading event 3000/10000
Now loading event 3100/10000
Now loading event 3200/10000
Now loading event 3300/10000
Now loading event 3400/100

In [6]:
# This file has 42986 data points. 
%time train3, labels3, weights3 = getDataFromJSON(path="data/train_partition3_data.json", earlyStop=10000)

Now loading event 0/10000
Now loading event 100/10000
Now loading event 200/10000
Now loading event 300/10000
Now loading event 400/10000
Now loading event 500/10000
Now loading event 600/10000
Now loading event 700/10000
Now loading event 800/10000
Now loading event 900/10000
Now loading event 1000/10000
Now loading event 1100/10000
Now loading event 1200/10000
Now loading event 1300/10000
Now loading event 1400/10000
Now loading event 1500/10000
Now loading event 1600/10000
Now loading event 1700/10000
Now loading event 1800/10000
Now loading event 1900/10000
Now loading event 2000/10000
Now loading event 2100/10000
Now loading event 2200/10000
Now loading event 2300/10000
Now loading event 2400/10000
Now loading event 2500/10000
Now loading event 2600/10000
Now loading event 2700/10000
Now loading event 2800/10000
Now loading event 2900/10000
Now loading event 3000/10000
Now loading event 3100/10000
Now loading event 3200/10000
Now loading event 3300/10000
Now loading event 3400/100

In [7]:
train2[0,0]

tensor([19.0601, 18.4611, 25.6917, 17.3464, 18.1081, 18.0577, 18.7077, 17.4748,
        17.8627, 16.9230, 22.6015, 22.5883, 18.8607, 19.9172, 23.7632, 21.3895,
        27.7625, 26.1885, 26.1678, 27.3103, 27.4704, 26.1131, 29.2025, 31.6294,
        28.8686, 27.6369, 26.0752, 20.9289, 19.3941, 17.8339, 23.0833, 19.8953,
        20.2367, 20.7759, 21.6280, 22.9786, 20.4450, 20.1092, 19.7991, 21.1444,
        16.3790, 20.6794, 16.9210, 18.0858, 16.4018, 16.2757, 17.2050, 19.5720,
        19.2548, 19.9467, 18.0555, 18.8806, 18.5460, 20.5339, 18.9813, 16.3843,
        18.5569, 17.3046, 17.9197, 14.8065])

In [35]:
# Define the network. Make sure to end with nn.Softmax activation
import torch.nn as nn
from skorch import NeuralNet

class simpleModel(nn.Module):
    def __init__(self, n, hidden_size, num_classes=5, drop1=.5):
        super().__init__() 
        self.n = n
        self.layer1 = []
        for _ in range(33):
            self.layer1.append(nn.Linear(60,n))
        self.layer2 = nn.Linear(n*33, hidden_size)
        self.layerout = nn.Linear(hidden_size, num_classes)
        #Define a RELU Activation unit
        self.relu = nn.ReLU()  
        self.smax = nn.Softmax(dim=1)
        self.drop = nn.Dropout(p=drop1)

    def forward(self, x):
        #Forward Propagate through the layers as defined above
        output = torch.empty((x.shape[0], 33, self.n))
        for i in range(33):
            output[:,i,:]=self.layer1[i](x[:, i, :])
            if i == 1: print(x[0,i,:])
        print(x)
        print('This is iffy')
        print(output)
        assert False
        y = self.relu( output.reshape(-1, 33*self.n))
        y = self.relu(self.layer2(y))
        y = self.smax(self.layerout(y))
        return y


In [36]:
model1 = simpleModel(1, 30)
model3 = simpleModel(3, 30)

In [52]:
def train(model, inputs, labels, weight, valSets, valLabels, valweight):
    # TODO: Is this right? How do I determine the weights here?
    weight = torch.Tensor(weight)
    lfc = nn.CrossEntropyLoss(weight=None)
#     valLabels = torch.tensor(valLabels, dtype=torch.int)
    #ideas
    # 1-(weight/np.sum(weight))
    # .2/weight - this one normalizes so that each class is responsible for 20% of the loss
    # 1/weight - this is a bit naive, but the classes with fewer items are weighted more.
    # 1/(weight+1) - makes sure we don't have any pesky zeroes
    # np.sum(weight)/weight if your learning rate is too low.
    
    # Hyperparameters
    batch = 256
    epochs = 10
    lr = 0.01
    
    # Start a dataloader object
    data = list(zip(inputs,labels))
    val = list(zip(valSets,valLabels))
    loader = DataLoader(data, batch_size = batch, num_workers=4)
    valLoader = DataLoader(val, batch_size = int(len(val)/4), num_workers=4)
    opt = torch.optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        model.train()
        batch_loss = []
        for (xtrain,ytrain) in loader:
            output = model(xtrain)
            loss = lfc(output,ytrain)
            print(loss)
            assert False
            opt.zero_grad()
            loss.backward()
            opt.step()
            batch_loss.append(loss.item())
        print(f'The training loss for epoch {epoch+1}/{epochs} was {np.mean(batch_loss)}')
        
        
        model.eval()
        balanced = [[],[],[],[],[]]
        batchLoss = []
        unbalanced = []
        
        for (xval,yval) in valLoader:
            output = model(xval)
            loss = lfc(output,yval)
            batchLoss.append(loss.item())
            corrects = yval.clone().detach() == torch.argmax(output)
            unbalanced.append(np.mean([1 if correct else 0 for correct in corrects.detach()]))
            
            # TODO: figure this one out.
            for i, ans in enumerate(yval):
                balanced[ans].append(corrects[i])
        
        balanced = [np.mean(i) for i in balanced]
        balancedAccuracy = np.mean(balanced)
        
        print(f'The total balanced accuracy for validation was {balancedAccuracy}')
        print(f'The unbalanced validation accuracy is {np.mean(unbalanced)}')
        print(f'The accuracy for each is {balanced}')           
        print(f'The validation loss was :   {epoch+1}/{epochs} was {np.mean(batchLoss)}')

            
        print(f'The validation loss was :   {epoch+1}/{epochs} was {np.mean(batchLoss)}')
    return model


In [53]:
print(model1)
model1 = train(
    model1,
    torch.cat((train1, train2), dim=0),
    labels1 + labels2,
    [weights1[i] + weights2[i] for i in range(5)],
    train3,
    labels3,
    weights3
)

print('================ here begins the training of the new model ===============')

model3 = train(
    model3,
    torch.cat((train1, train2), dim=0),
    labels1 + labels2,
    [weights1[i] + weights2[i] for i in range(5)],
    train3,
    labels3,
    weights3
)

simpleModel(
  (layer2): Linear(in_features=33, out_features=30, bias=True)
  (layerout): Linear(in_features=30, out_features=5, bias=True)
  (relu): ReLU()
  (smax): Softmax(dim=1)
  (drop): Dropout(p=0.5, inplace=False)
)
tensor([2.5324e+10, 2.5299e+10, 2.5383e+10, 2.5641e+10, 2.5717e+10, 2.5705e+10,
        2.5911e+10, 2.5993e+10, 2.6150e+10, 2.6238e+10, 2.6213e+10, 2.6242e+10,
        2.6259e+10, 2.6233e+10, 2.6227e+10, 2.6129e+10, 2.6101e+10, 2.6040e+10,
        2.6038e+10, 2.5739e+10, 2.5711e+10, 2.5693e+10, 2.5609e+10, 2.5570e+10,
        2.5561e+10, 2.5571e+10, 2.5564e+10, 2.5549e+10, 2.5484e+10, 2.5468e+10,
        2.5498e+10, 2.5502e+10, 2.5365e+10, 2.5313e+10, 2.5229e+10, 2.5170e+10,
        2.5025e+10, 2.4924e+10, 2.4778e+10, 2.4476e+10, 2.4336e+10, 2.4267e+10,
        2.4127e+10, 2.4175e+10, 2.4057e+10, 2.4005e+10, 2.3994e+10, 2.4100e+10,
        2.4046e+10, 2.4052e+10, 2.4056e+10, 2.4023e+10, 2.4093e+10, 2.4187e+10,
        2.4107e+10, 2.4106e+10, 2.4065e+10, 2.4020e+10, 

AssertionError: 

In [56]:
print(model3.parameters())
feature_extraction = [child for child in model1.children()]
print(feature_extraction[1])

<generator object Module.parameters at 0x7f59a5bdd820>
Linear(in_features=30, out_features=5, bias=True)
